# Transformer チュートリアル

このノートブックでは、自作Transformerの使い方を解説します。

## 目次

1. [環境セットアップ](#1-環境セットアップ)
2. [アーキテクチャ概要](#2-アーキテクチャ概要)
3. [モデルの作成](#3-モデルの作成)
4. [パラメータの理解](#4-パラメータの理解)
5. [データの準備](#5-データの準備)
6. [学習方法](#6-学習方法)
7. [推論（生成）方法](#7-推論生成方法)
8. [実践例：コピータスク](#8-実践例コピータスク)
9. [実践例：加算タスク](#9-実践例加算タスク)
10. [Tips & トラブルシューティング](#10-tips--トラブルシューティング)

---
## 1. 環境セットアップ

In [ ]:
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# 日本語フォントの設定（macOS）
import matplotlib
matplotlib.rcParams['font.family'] = 'Hiragino Sans'
matplotlib.rcParams['axes.unicode_minus'] = False

# デバイスの設定
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(f"Using device: {device}")

In [ ]:
# 自作Transformerのインポート
from src.transformer import Transformer, count_parameters
from src.attention import SelfAttention, MultiHeadAttention
from src.encoder import TransformerEncoder
from src.decoder import TransformerDecoder
from src.position_encoding import PositionalEncoding
from src.feed_forward import FeedForward

print("All modules imported successfully!")

---
## 2. アーキテクチャ概要

Transformerは以下のコンポーネントで構成されています：

```
Transformer
├── Encoder
│   ├── Token Embedding
│   ├── Positional Encoding
│   └── EncoderLayer × N
│       ├── Multi-Head Self-Attention
│       ├── Add & Norm
│       ├── Feed Forward Network
│       └── Add & Norm
│
├── Decoder
│   ├── Token Embedding
│   ├── Positional Encoding
│   └── DecoderLayer × N
│       ├── Masked Multi-Head Self-Attention
│       ├── Add & Norm
│       ├── Multi-Head Cross-Attention
│       ├── Add & Norm
│       ├── Feed Forward Network
│       └── Add & Norm
│
└── Output Layer (Linear)
```

### ファイル構成

| ファイル | 内容 |
|---------|------|
| `src/attention.py` | SelfAttention, MultiHeadAttention |
| `src/position_encoding.py` | PositionalEncoding |
| `src/feed_forward.py` | FeedForward |
| `src/encoder.py` | EncoderLayer, Encoder, TransformerEncoder |
| `src/decoder.py` | DecoderLayer, Decoder, TransformerDecoder |
| `src/transformer.py` | Transformer（完全なモデル） |

---
## 3. モデルの作成

### 3.1 基本的な使い方

In [ ]:
# 最もシンプルなモデル作成
model = Transformer(
    src_vocab_size=100,  # ソース語彙サイズ
    tgt_vocab_size=100,  # ターゲット語彙サイズ
)

print(f"Model created!")
print(f"Parameters: {count_parameters(model):,}")

In [ ]:
# カスタムパラメータでモデル作成
model = Transformer(
    src_vocab_size=100,       # ソース語彙サイズ
    tgt_vocab_size=100,       # ターゲット語彙サイズ
    d_model=128,              # 埋め込み次元
    num_heads=4,              # Attentionヘッド数
    num_encoder_layers=3,     # Encoder層数
    num_decoder_layers=3,     # Decoder層数
    d_ff=512,                 # FFN中間層の次元
    max_len=512,              # 最大シーケンス長
    dropout=0.1,              # ドロップアウト率
)

print(f"Parameters: {count_parameters(model):,}")

# GPUに移動
model = model.to(device)

### 3.2 モデル構造の確認

In [ ]:
# モデル構造を表示
print(model)

In [ ]:
# パラメータ数の内訳
def count_params_by_module(model):
    """モジュールごとのパラメータ数を表示"""
    total = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num = param.numel()
            total += num
            print(f"{name}: {num:,}")
    print(f"\nTotal: {total:,}")

# 簡易版：主要コンポーネントごと
# 注意: Transformerの構造は src_embedding, tgt_embedding, encoder, decoder, output_projection
embedding_params = sum(p.numel() for p in model.src_embedding.parameters()) + \
                   sum(p.numel() for p in model.tgt_embedding.parameters())
encoder_params = sum(p.numel() for p in model.encoder.parameters())
decoder_params = sum(p.numel() for p in model.decoder.parameters())
output_params = sum(p.numel() for p in model.output_projection.parameters())

print("パラメータ配分:")
print(f"  Embedding: {embedding_params:,} ({embedding_params/count_parameters(model)*100:.1f}%)")
print(f"  Encoder:   {encoder_params:,} ({encoder_params/count_parameters(model)*100:.1f}%)")
print(f"  Decoder:   {decoder_params:,} ({decoder_params/count_parameters(model)*100:.1f}%)")
print(f"  Output:    {output_params:,} ({output_params/count_parameters(model)*100:.1f}%)")

---
## 4. パラメータの理解

### 4.1 主要パラメータ

| パラメータ | 説明 | 推奨値 |
|-----------|------|--------|
| `src_vocab_size` | 入力の語彙サイズ | タスク依存 |
| `tgt_vocab_size` | 出力の語彙サイズ | タスク依存 |
| `d_model` | 埋め込み次元 | 64, 128, 256, 512 |
| `num_heads` | Attentionヘッド数 | d_modelの約数 |
| `num_encoder_layers` | Encoder層数 | 2-6 |
| `num_decoder_layers` | Decoder層数 | 2-6 |
| `d_ff` | FFN中間層次元 | d_model × 4 |
| `max_len` | 最大シーケンス長 | 512, 1024, 5000 |
| `dropout` | ドロップアウト率 | 0.1 |

### 4.2 パラメータ選択のガイドライン

In [ ]:
# タスク規模別の推奨設定

# 小規模タスク（コピー、簡単な変換）
small_config = {
    'd_model': 64,
    'num_heads': 4,
    'num_encoder_layers': 2,
    'num_decoder_layers': 2,
    'd_ff': 256,
}

# 中規模タスク（算術、パターン認識）
medium_config = {
    'd_model': 128,
    'num_heads': 4,
    'num_encoder_layers': 3,
    'num_decoder_layers': 3,
    'd_ff': 512,
}

# 大規模タスク（複雑な変換）
large_config = {
    'd_model': 256,
    'num_heads': 8,
    'num_encoder_layers': 4,
    'num_decoder_layers': 4,
    'd_ff': 1024,
}

# パラメータ数を比較
for name, config in [('Small', small_config), ('Medium', medium_config), ('Large', large_config)]:
    m = Transformer(src_vocab_size=100, tgt_vocab_size=100, **config)
    print(f"{name}: {count_parameters(m):,} parameters")

### 4.3 重要な制約

```python
# d_model は num_heads で割り切れる必要がある
assert d_model % num_heads == 0

# 例: d_model=128, num_heads=4 → d_k = 128/4 = 32 (OK)
# 例: d_model=128, num_heads=5 → エラー！
```

---
## 5. データの準備

### 5.1 特殊トークン

Transformerでは以下の特殊トークンが必要です：

| トークン | 説明 | 用途 |
|---------|------|------|
| `PAD` | パディング | バッチ内のシーケンス長を揃える |
| `START` | 開始トークン | Decoderの最初の入力 |
| `END` | 終了トークン | 生成の終了を示す |

In [ ]:
# 特殊トークンの定義
PAD_IDX = 0   # パディング
START_IDX = 1 # 開始トークン
END_IDX = 2   # 終了トークン

# 実際のトークンは3から始まる
# 例: 数字0-9を使う場合
DIGIT_OFFSET = 3  # 数字0 → トークン3, 数字9 → トークン12

### 5.2 入力データの形式

```python
# 入力の形式
src     = [batch_size, src_seq_len]     # ソースシーケンス
tgt_in  = [batch_size, tgt_seq_len]     # ターゲット入力（STARTで始まる）
tgt_out = [batch_size, tgt_seq_len]     # ターゲット出力（ENDで終わる）

# 例: "12+34=46" をコピーするタスク
src     = [1, 2, 3, 4]           # 入力: [1, 2, 3, 4]
tgt_in  = [START, 1, 2, 3, 4]    # Decoder入力
tgt_out = [1, 2, 3, 4, END]      # 正解ラベル
```

In [ ]:
def create_sample_data(batch_size=4, seq_len=5):
    """
    サンプルデータを作成する例
    
    Returns:
        src: [batch_size, seq_len] - ソースシーケンス
        tgt_in: [batch_size, seq_len+1] - Decoder入力（STARTで始まる）
        tgt_out: [batch_size, seq_len+1] - 正解ラベル（ENDで終わる）
    """
    # ランダムなソースシーケンス（3-12の範囲、数字0-9に相当）
    src = torch.randint(DIGIT_OFFSET, DIGIT_OFFSET + 10, (batch_size, seq_len))
    
    # ターゲット入力: [START, src...]
    tgt_in = torch.cat([
        torch.full((batch_size, 1), START_IDX),
        src
    ], dim=1)
    
    # ターゲット出力: [src..., END]
    tgt_out = torch.cat([
        src,
        torch.full((batch_size, 1), END_IDX)
    ], dim=1)
    
    return src, tgt_in, tgt_out

# テスト
src, tgt_in, tgt_out = create_sample_data(batch_size=2, seq_len=4)
print("Source:     ", src[0].tolist())
print("Target In:  ", tgt_in[0].tolist())
print("Target Out: ", tgt_out[0].tolist())

### 5.3 パディングの処理

バッチ内でシーケンス長が異なる場合、パディングが必要です。

In [ ]:
def pad_sequences(sequences, pad_value=PAD_IDX):
    """
    可変長シーケンスをパディング
    
    Args:
        sequences: リスト of リスト（各シーケンス）
        pad_value: パディングに使う値
    
    Returns:
        padded: [batch_size, max_len] のテンソル
    """
    max_len = max(len(seq) for seq in sequences)
    padded = torch.full((len(sequences), max_len), pad_value, dtype=torch.long)
    
    for i, seq in enumerate(sequences):
        padded[i, :len(seq)] = torch.tensor(seq)
    
    return padded

# テスト
sequences = [
    [3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12],
]
padded = pad_sequences(sequences)
print("Padded sequences:")
print(padded)

---
## 6. 学習方法

### 6.1 基本的な学習ループ

In [ ]:
def train_step(model, src, tgt_in, tgt_out, optimizer, criterion):
    """
    1ステップの学習
    
    Args:
        model: Transformerモデル
        src: ソースシーケンス [batch, src_len]
        tgt_in: ターゲット入力 [batch, tgt_len]
        tgt_out: 正解ラベル [batch, tgt_len]
        optimizer: オプティマイザ
        criterion: 損失関数
    
    Returns:
        loss: 損失値
        accuracy: 精度
    """
    model.train()
    
    # 順伝播
    logits = model(src, tgt_in)  # [batch, tgt_len, vocab_size]
    
    # 損失計算
    loss = criterion(
        logits.reshape(-1, logits.size(-1)),  # [batch*tgt_len, vocab_size]
        tgt_out.reshape(-1)                    # [batch*tgt_len]
    )
    
    # 精度計算（PADを除く）
    predictions = logits.argmax(dim=-1)
    mask = tgt_out != PAD_IDX
    accuracy = ((predictions == tgt_out) & mask).sum().float() / mask.sum().float()
    
    # 逆伝播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item(), accuracy.item()

In [ ]:
def train_model(model, data_generator, num_epochs=100, lr=0.001, verbose=True):
    """
    モデルを学習する
    
    Args:
        model: Transformerモデル
        data_generator: データ生成関数 (batch_size) -> (src, tgt_in, tgt_out)
        num_epochs: エポック数
        lr: 学習率
        verbose: 進捗表示
    
    Returns:
        losses: 損失の履歴
        accuracies: 精度の履歴
    """
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    
    losses = []
    accuracies = []
    
    iterator = range(num_epochs)
    if verbose:
        iterator = tqdm(iterator, desc="Training")
    
    for epoch in iterator:
        # データ生成
        src, tgt_in, tgt_out = data_generator(batch_size=64)
        src = src.to(device)
        tgt_in = tgt_in.to(device)
        tgt_out = tgt_out.to(device)
        
        # 学習ステップ
        loss, acc = train_step(model, src, tgt_in, tgt_out, optimizer, criterion)
        
        losses.append(loss)
        accuracies.append(acc)
        
        if verbose:
            iterator.set_postfix(loss=f"{loss:.4f}", acc=f"{acc:.4f}")
    
    return losses, accuracies


def plot_training(losses, accuracies, title="Training"):
    """学習曲線を描画"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    ax1.plot(losses)
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title(f'{title} - Loss')
    ax1.grid(True, alpha=0.3)
    
    ax2.plot(accuracies)
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title(f'{title} - Accuracy')
    ax2.set_ylim(0, 1.05)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

### 6.2 学習のポイント

1. **学習率**: 0.0001 ~ 0.001 が一般的
2. **エポック数**: タスクの複雑さに依存（1000~10000）
3. **バッチサイズ**: 32~128
4. **勾配クリッピング**: 大きな勾配を制限する

In [ ]:
# 勾配クリッピングの例
def train_step_with_clip(model, src, tgt_in, tgt_out, optimizer, criterion, max_norm=1.0):
    """勾配クリッピング付きの学習ステップ"""
    model.train()
    
    logits = model(src, tgt_in)
    loss = criterion(
        logits.reshape(-1, logits.size(-1)),
        tgt_out.reshape(-1)
    )
    
    optimizer.zero_grad()
    loss.backward()
    
    # 勾配クリッピング
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)
    
    optimizer.step()
    
    return loss.item()

---
## 7. 推論（生成）方法

学習済みモデルを使って、新しい入力から出力を生成します。

### 7.1 Greedy Decode（貪欲法）

In [ ]:
def greedy_decode_example(model, src, max_len=10):
    """
    Greedy Decodeの例
    
    各ステップで最も確率の高いトークンを選択します。
    """
    model.eval()
    
    with torch.no_grad():
        # モデルの組み込みメソッドを使用
        generated = model.greedy_decode(
            src,
            max_len=max_len,
            start_token_id=START_IDX,
            end_token_id=END_IDX
        )
    
    return generated

# 使い方
print("Greedy Decode:")
print("  最も確率の高いトークンを毎回選択")
print("  決定的な出力（同じ入力→同じ出力）")
print("  高速")

### 7.2 サンプリング生成

In [ ]:
def sampling_decode_example(model, src, max_len=10, temperature=1.0, top_k=None, top_p=None):
    """
    サンプリング生成の例
    
    確率分布からサンプリングして次のトークンを決定します。
    
    Args:
        temperature: 温度パラメータ
            - < 1.0: よりシャープ（確定的）
            - = 1.0: オリジナルの分布
            - > 1.0: よりフラット（ランダム）
        top_k: 上位k個のトークンからのみサンプリング
        top_p: 累積確率がp以下のトークンからサンプリング（Nucleus Sampling）
    """
    model.eval()
    
    with torch.no_grad():
        generated = model.generate(
            src,
            max_len=max_len,
            start_token_id=START_IDX,
            end_token_id=END_IDX,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p
        )
    
    return generated

# パラメータの説明
print("サンプリングパラメータ:")
print()
print("Temperature:")
print("  0.5 - より確定的（同じ出力が多い）")
print("  1.0 - オリジナル分布")
print("  2.0 - より多様（ランダム）")
print()
print("Top-K:")
print("  k=1  - Greedy Decode と同じ")
print("  k=10 - 上位10トークンからサンプリング")
print()
print("Top-P (Nucleus):")
print("  p=0.9 - 累積確率90%までのトークンからサンプリング")

---
## 8. 実践例：コピータスク

入力をそのまま出力するタスクで、モデルの基本動作を確認します。

In [ ]:
# コピータスク用のデータ生成
def generate_copy_data(batch_size, seq_len=5):
    """
    コピータスクのデータ生成
    入力: [3, 5, 7, 4, 6]
    出力: [3, 5, 7, 4, 6]
    """
    # ランダムなシーケンス（3-12）
    src = torch.randint(3, 13, (batch_size, seq_len))
    
    # ターゲット入力: [START, src...]
    tgt_in = torch.cat([
        torch.full((batch_size, 1), START_IDX),
        src
    ], dim=1)
    
    # ターゲット出力: [src..., END]
    tgt_out = torch.cat([
        src,
        torch.full((batch_size, 1), END_IDX)
    ], dim=1)
    
    return src, tgt_in, tgt_out

# テスト
src, tgt_in, tgt_out = generate_copy_data(2)
print("Copy Task Example:")
print(f"  Input:  {src[0].tolist()}")
print(f"  Output: {[t for t in tgt_out[0].tolist() if t != END_IDX]}")

In [ ]:
# モデル作成
copy_model = Transformer(
    src_vocab_size=15,
    tgt_vocab_size=15,
    d_model=64,
    num_heads=4,
    num_encoder_layers=2,
    num_decoder_layers=2,
    d_ff=256,
).to(device)

print(f"Parameters: {count_parameters(copy_model):,}")

# 学習
losses, accs = train_model(
    copy_model, 
    generate_copy_data, 
    num_epochs=500, 
    lr=0.001
)

plot_training(losses, accs, "Copy Task")

In [ ]:
# テスト
print("\nCopy Task Test Results:")
print("=" * 50)

copy_model.eval()
src, _, _ = generate_copy_data(5)
src_device = src.to(device)

with torch.no_grad():
    generated = copy_model.greedy_decode(
        src_device, 
        max_len=7,
        start_token_id=START_IDX, 
        end_token_id=END_IDX
    )

for i in range(5):
    input_seq = src[i].tolist()
    output_seq = [t for t in generated[i].tolist() if t not in [START_IDX, END_IDX, PAD_IDX]]
    match = input_seq == output_seq
    mark = "✓" if match else "✗"
    print(f"{mark} Input: {input_seq} -> Output: {output_seq}")

---
## 9. 実践例：加算タスク

2桁の加算（例: 23 + 45 = 68）を学習します。

In [ ]:
# 加算タスク用の定数
ADD_PAD = 0
ADD_START = 1
ADD_END = 2
ADD_PLUS = 3
ADD_DIGIT_OFFSET = 4  # 数字0 → トークン4

def num_to_tokens(n, num_digits):
    """数字をトークン列に変換"""
    digits = [int(d) + ADD_DIGIT_OFFSET for d in str(n).zfill(num_digits)]
    return digits

def tokens_to_num(tokens):
    """トークン列を数字に変換"""
    digits = [t - ADD_DIGIT_OFFSET for t in tokens if ADD_DIGIT_OFFSET <= t <= ADD_DIGIT_OFFSET + 9]
    if not digits:
        return 0
    return int(''.join(map(str, digits)))

def generate_addition_data(batch_size, num_digits=2):
    """
    加算タスクのデータ生成
    例: 23 + 45 = 68
    入力: [2, 3, +, 4, 5]
    出力: [6, 8]
    """
    src_list = []
    tgt_in_list = []
    tgt_out_list = []
    
    max_num = 10 ** num_digits - 1
    result_digits = num_digits + 1  # 桁上がり用
    
    for _ in range(batch_size):
        a = torch.randint(0, max_num + 1, (1,)).item()
        b = torch.randint(0, max_num + 1, (1,)).item()
        result = a + b
        
        # ソース: a + b
        src_tokens = num_to_tokens(a, num_digits) + [ADD_PLUS] + num_to_tokens(b, num_digits)
        
        # ターゲット: result
        result_tokens = num_to_tokens(result, result_digits)
        
        src_list.append(torch.tensor(src_tokens))
        tgt_in_list.append(torch.tensor([ADD_START] + result_tokens))
        tgt_out_list.append(torch.tensor(result_tokens + [ADD_END]))
    
    src = torch.stack(src_list)
    tgt_in = torch.stack(tgt_in_list)
    tgt_out = torch.stack(tgt_out_list)
    
    return src, tgt_in, tgt_out

# テスト
src, tgt_in, tgt_out = generate_addition_data(3)
print("Addition Task Examples:")
for i in range(3):
    src_t = src[i].tolist()
    plus_pos = src_t.index(ADD_PLUS)
    a = tokens_to_num(src_t[:plus_pos])
    b = tokens_to_num(src_t[plus_pos+1:])
    result = tokens_to_num([t for t in tgt_out[i].tolist() if t != ADD_END])
    print(f"  {a} + {b} = {result}")

In [ ]:
# 加算モデル作成
add_model = Transformer(
    src_vocab_size=14,  # PAD, START, END, +, 0-9
    tgt_vocab_size=14,
    d_model=128,
    num_heads=4,
    num_encoder_layers=3,
    num_decoder_layers=3,
    d_ff=512,
).to(device)

print(f"Parameters: {count_parameters(add_model):,}")

# 学習（2桁の加算）
def gen_add_2digit(batch_size):
    return generate_addition_data(batch_size, num_digits=2)

# 注意: 加算タスクは学習に時間がかかります
# 完全な精度には10000エポック必要
add_losses, add_accs = train_model(
    add_model, 
    gen_add_2digit, 
    num_epochs=3000,  # 短めのデモ
    lr=0.0005
)

plot_training(add_losses, add_accs, "Addition Task (2 digits)")

In [ ]:
# テスト
print("\nAddition Task Test Results:")
print("=" * 50)

add_model.eval()
correct = 0
total = 20

for _ in range(total):
    src, _, _ = generate_addition_data(1, num_digits=2)
    src_device = src.to(device)
    
    with torch.no_grad():
        generated = add_model.greedy_decode(
            src_device,
            max_len=5,
            start_token_id=ADD_START,
            end_token_id=ADD_END
        )
    
    src_t = src[0].tolist()
    plus_pos = src_t.index(ADD_PLUS)
    a = tokens_to_num(src_t[:plus_pos])
    b = tokens_to_num(src_t[plus_pos+1:])
    expected = a + b
    
    gen_tokens = [t for t in generated[0].tolist() if t not in [ADD_START, ADD_END, ADD_PAD]]
    predicted = tokens_to_num(gen_tokens)
    
    match = expected == predicted
    if match:
        correct += 1
    mark = "✓" if match else "✗"
    print(f"{mark} {a:2d} + {b:2d} = {predicted:3d}  (expected: {expected})")

print(f"\nAccuracy: {correct}/{total} = {correct/total:.1%}")
print("\n注: 100%精度には10000エポックの学習が必要です")

---
## 10. Tips & トラブルシューティング

### 10.1 よくある問題と解決策

| 問題 | 原因 | 解決策 |
|------|------|--------|
| 損失が下がらない | 学習率が高すぎる/低すぎる | 学習率を調整（0.0001~0.001） |
| 精度が上がらない | エポック不足 | より多くのエポックで学習 |
| 過学習 | モデルが大きすぎる | dropout増加、モデル縮小 |
| メモリ不足 | バッチサイズ/モデルが大きい | バッチサイズ縮小、d_model縮小 |
| 出力が同じトークンの繰り返し | 学習不足 | エポック増加 |

### 10.2 デバッグのコツ

In [ ]:
# 1. 入出力の形状確認
def debug_shapes(model, src, tgt_in):
    """各層の出力形状を確認"""
    model.eval()
    with torch.no_grad():
        print(f"Input shapes:")
        print(f"  src: {src.shape}")
        print(f"  tgt_in: {tgt_in.shape}")
        
        # Encoder出力
        enc_output = model.encode(src)
        print(f"\nEncoder output: {enc_output.shape}")
        
        # Decoder出力
        logits = model(src, tgt_in)
        print(f"Decoder output (logits): {logits.shape}")

# テスト
src, tgt_in, _ = generate_copy_data(2, seq_len=4)
debug_shapes(copy_model, src.to(device), tgt_in.to(device))

In [ ]:
# 2. Attention重みの可視化
def visualize_attention(model, src, tgt_in):
    """Attention重みを可視化（簡易版）"""
    model.eval()
    
    # フックを使ってAttention重みを取得
    attention_weights = []
    
    def hook_fn(module, input, output):
        if isinstance(output, tuple) and len(output) == 2:
            attention_weights.append(output[1].detach().cpu())
    
    # 最初のEncoder層のAttentionにフックを登録
    # 構造: model.encoder.layers[0].self_attention
    handle = model.encoder.layers[0].self_attention.register_forward_hook(hook_fn)
    
    with torch.no_grad():
        _ = model(src, tgt_in)
    
    handle.remove()
    
    if attention_weights:
        # 最初のサンプル、最初のヘッドのAttention重み
        attn = attention_weights[0][0, 0].numpy()
        
        plt.figure(figsize=(6, 5))
        plt.imshow(attn, cmap='Blues')
        plt.colorbar()
        plt.xlabel('Key Position')
        plt.ylabel('Query Position')
        plt.title('Encoder Self-Attention (Head 0)')
        plt.show()
    else:
        print("Attention weights not captured")

# テスト
src, tgt_in, _ = generate_copy_data(1, seq_len=5)
visualize_attention(copy_model, src.to(device), tgt_in.to(device))

### 10.3 パフォーマンス改善のヒント

1. **学習率スケジューリング**

In [ ]:
# Warmupスケジューラの例
def get_lr_scheduler(optimizer, warmup_steps=1000, d_model=128):
    """
    Transformer論文のスケジューラ
    lr = d_model^(-0.5) * min(step^(-0.5), step * warmup^(-1.5))
    """
    def lr_lambda(step):
        step = max(step, 1)
        return d_model ** (-0.5) * min(step ** (-0.5), step * warmup_steps ** (-1.5))
    
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# 使用例
# optimizer = torch.optim.Adam(model.parameters(), lr=1.0)
# scheduler = get_lr_scheduler(optimizer)
# 
# for epoch in range(num_epochs):
#     train_step(...)
#     scheduler.step()

2. **Early Stopping**

In [ ]:
class EarlyStopping:
    """Early Stoppingの実装"""
    def __init__(self, patience=10, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.should_stop = False
    
    def __call__(self, loss):
        if self.best_loss is None:
            self.best_loss = loss
        elif loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.should_stop = True
        else:
            self.best_loss = loss
            self.counter = 0
        
        return self.should_stop

# 使用例
# early_stopping = EarlyStopping(patience=10)
# 
# for epoch in range(num_epochs):
#     loss = train_step(...)
#     if early_stopping(loss):
#         print("Early stopping!")
#         break

### 10.4 モデルの保存と読み込み

In [ ]:
# モデルの保存
def save_model(model, path):
    """モデルを保存"""
    torch.save({
        'model_state_dict': model.state_dict(),
        'config': {
            'src_vocab_size': model.src_embedding.num_embeddings,
            'tgt_vocab_size': model.tgt_embedding.num_embeddings,
            'd_model': model.d_model,
            # 他の設定も保存可能
        }
    }, path)
    print(f"Model saved to {path}")

# モデルの読み込み
def load_model(path, device='cpu'):
    """モデルを読み込み"""
    checkpoint = torch.load(path, map_location=device)
    config = checkpoint['config']
    
    model = Transformer(**config)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.to(device)
    model.eval()
    
    print(f"Model loaded from {path}")
    return model

# 使用例（コメントアウト）
# save_model(copy_model, 'copy_model.pt')
# loaded_model = load_model('copy_model.pt', device)

---
## まとめ

### 基本的な使い方フロー

```python
# 1. モデル作成
model = Transformer(
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    d_model=128,
    num_heads=4,
    num_encoder_layers=3,
    num_decoder_layers=3,
).to(device)

# 2. データ準備
src, tgt_in, tgt_out = generate_data(batch_size)

# 3. 学習
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

for epoch in range(num_epochs):
    logits = model(src, tgt_in)
    loss = criterion(logits.reshape(-1, vocab_size), tgt_out.reshape(-1))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# 4. 推論
model.eval()
with torch.no_grad():
    generated = model.greedy_decode(src, max_len=10, start_token_id=1, end_token_id=2)
```

### 重要なポイント

1. **学習時間は重要**: 複雑なタスクには十分なエポック数が必要
2. **シンプルな表現を使う**: 余計な前処理は不要なことが多い
3. **条件を揃えて比較する**: 公平な比較のため、パラメータを統一
4. **小さく始める**: まずは小さなモデルで動作確認

### 関連ノートブック

- `01_self_attention_demo.ipynb`: Self-Attentionの詳細
- `02_multi_head_attention_demo.ipynb`: Multi-Head Attentionの詳細
- `07_transformer_demo.ipynb`: Transformerの構造解説
- `08_diverse_tasks_demo.ipynb`: 様々なタスクでの学習例
- `09_addition_improvement.ipynb`: 加算タスクの改善実験